<a href="https://colab.research.google.com/github/19WH1A0501/Stock-Prediction-Using-LSTM-and-ML-Methods/blob/main/new_major_wipro_infosys_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 KB 1.2 MB/s eta 0:00:00


In [ ]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

In [ ]:
query = "infosys (#Infosys, OR #Infy, OR #InfosysCareers, OR #InfyNite, OR #InfyGold, OR #InfyPulse, OR #InfyMaker, OR #InfosysLeadership, OR #InfosysDigital, OR #InfyTech, OR #InfosysCSR, OR #InfosysFoundation) lang:en until:2022-12-31 since:2017-01-01" 
tweets = []
limit = 100000
i = 0
for tweet in sntwitter.TwitterSearchScraper(query).get_items():
    print(i, pd.to_datetime(tweet.date), tweet.content[:10])
    tweets.append([tweet.date, tweet.content])
    i += 1


<ipython-input-5-46deba547c86>:6: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  print(i, pd.to_datetime(tweet.date), tweet.content[:10])
<ipython-input-5-46deba547c86>:7: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tweets.append([tweet.date, tweet.content])


Streaming output truncated to the last 5000 lines.
73813 2017-04-13 05:08:25+00:00 #Infosys Q
73814 2017-04-13 05:06:21+00:00 #Infosys c
73815 2017-04-13 05:06:18+00:00 FY18 guida
73816 2017-04-13 05:06:18+00:00 Infosys de
73817 2017-04-13 05:05:39+00:00 🕙 10:35 AM
73818 2017-04-13 05:05:32+00:00 #Infosys n
73819 2017-04-13 05:05:15+00:00 #Infosys n
73820 2017-04-13 05:04:38+00:00 .@CNBC_Awa
73821 2017-04-13 05:03:52+00:00 IT stocks 
73822 2017-04-13 05:03:29+00:00 Infosys to
73823 2017-04-13 05:03:15+00:00 The last t
73824 2017-04-13 05:03:11+00:00 Salute to 
73825 2017-04-13 05:02:52+00:00 .@CNBC_Awa
73826 2017-04-13 05:02:14+00:00 Wishing yo
73827 2017-04-13 05:01:52+00:00 #Infosys Q
73828 2017-04-13 05:01:34+00:00 .@CNBC_Awa
73829 2017-04-13 05:01:19+00:00 #Infosys R
73830 2017-04-13 05:00:42+00:00 .@CNBC_Awa
73831 2017-04-13 05:00:32+00:00 Global #In
73832 2017-04-13 05:00:14+00:00 🕙 10:30 AM
73833 2017-04-13 05:00:01+00:00 #PM #Modi 
73834 2017-04-13 04:58:52+00:00 Equity Mar
738

In [ ]:
import pandas as pd

In [ ]:
tweets = pd.DataFrame(tweets)

In [ ]:
tweets.to_csv('infosys_tweets.csv', index=False)

In [ ]:
# mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', None)

In [ ]:
!pip install demoji
import demoji
import re
import string
from nltk.corpus import stopwords,wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from nltk import pos_tag
import attr
import nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.9/42.9 kB 2.3 MB/s eta 0:00:00


In [ ]:
nltk.download("stopwords")
nltk.download('omw-1.4')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
def get_wordnet_pos(word):
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)
    

In [ ]:
def clean_text(text):
    tk = TweetTokenizer(preserve_case=False, strip_handles=True,reduce_len=True) 
    lemmatizer = WordNetLemmatizer()  
    stop_words = stopwords.words('english') + ["i'll","i'm", "should", "could"]
    negative_verbs = [ "shan't",'shouldn',"shouldn't",'wasn','weren','won','wouldn','aren','couldn','didn','doesn','hadn','hasn','haven','isn','ma','mightn','mustn',"mustn't",'needn',"needn't","wouldn't","won't","weren't","wasn't","couldn","not","nor","no","mightn't","isn't","haven't","hadn't","hasn't","didn't","doesn't","aren't","don't","couldn't","never"]
    stop_words =[word for word in stop_words if word not in negative_verbs ] 
    
    lower_tweet = text.lower() 
    tweet = re.sub(r"[#$]"," ",lower_tweet)
    tweet = re.sub(r"https?:\/\/.*[\r\n]*"," ", tweet)
    tweet = demoji.replace_with_desc(tweet)
    tweet = re.sub(r"[0-9]|-->","",tweet)
    tweet = tk.tokenize(tweet)
    tweet = [lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in tweet if word not in stop_words and word not in string.punctuation and len(word)>2 and "." not in word]
    tweet = " ".join(tweet)
    
    return tweet

In [ ]:
tweets = pd.read_csv("/content/tcs_tweets18.csv", engine='python')

In [ ]:
tweets

,0,1
0,2018-12-30 19:58:21+00:00,"Cultivating Crime: How Colorado became a Major Exporter of Black Market Marijuana https://t.co/ZxqvxcSMKr @TerrapinCare #TCS wants to lock up home growers? ""One of the mistakes made in CO and other states is allowing for home cultivation,"" Chris Woods, founder of Terrapin Care"
1,2018-12-30 18:51:14+00:00,Rahul from TCS grabbed the Man of the Match in the TCS-Atos match\n#puneITCup2018 #PuneSports #PrathamSports #CorporateCricket #TCS #Atos #CSR #AnnualSportsEvent #CorporateSports #PuneITCupLeagueMatches #ManOfTheMatch https://t.co/Xn9ACOxsac
2,2018-12-30 18:18:52+00:00,@siasatpk @MuradSaeedPTI I'm from Karak one week before I was there in post office I shocked to see #TCS using Pakistan post for sending there couriers. @MuradSaeedPTI keep it up
3,2018-12-30 17:40:00+00:00,#TCS - TCS (Tata Consultancy Services Limited) #SELL BELOW 1890 - TradingView - https://t.co/YajWHOu0qs
4,2018-12-30 17:32:09+00:00,TCS (Tata Consultancy Services Limited) #SELL BELOW 1890 - #TCS chart https://t.co/lnKukdwRG9
...,...,...
9993,2018-01-01 07:26:31+00:00,5/6 RAKESHPATEL-Q-JAN18 out the changes at https://t.co/fAN4ZYQN8x #ACCELYA #AJANTPHARM #IGL #BYKE #TCS #MONSANTO #VSTIND #MAYURUNIQ
9994,2018-01-01 07:23:47+00:00,1/2 RAKESHPATEL-Q-JAN18 Another new portfolio - https://t.co/fAN4ZYQN8x - Created by Rakesh-Patel #ACCELYA #AJANTPHARM #IGL #BYKE #TCS
9995,2018-01-01 07:21:34+00:00,It is your opportunity to write a beautiful story for yourself.\nThe @InterviewReady_ team wishes you a very Happy New Year. #happynewyear #2018 #interviewready #TCS #careercoach https://t.co/tVJ4omi33t
9996,2018-01-01 06:40:20+00:00,2K18 Here we are\nWe made it.\nJah mi dey grateful\n#our year of divine open doors\n#TCS @ Satellite… https://t.co/QM1DegF3r5


In [ ]:
tweets['1'] = tweets['1'].fillna('')

In [ ]:
tweets.isna().sum()

0    0
1    0
dtype: int64

In [ ]:

# tweets = tweets[:100]
tweets['cleaned'] = tweets["1"].apply(lambda row:clean_text(row))
tweets.to_csv("CleanedNTweets_tcs18.csv",index=False)

In [ ]:
df = pd.read_csv('CleanedNTweets_tcs18.csv')

In [ ]:
df

,0,1,cleaned
0,2018-12-30 19:58:21+00:00,"Cultivating Crime: How Colorado became a Major Exporter of Black Market Marijuana https://t.co/ZxqvxcSMKr @TerrapinCare #TCS wants to lock up home growers? ""One of the mistakes made in CO and other states is allowing for home cultivation,"" Chris Woods, founder of Terrapin Care",cultivate crime colorado become major exporter black market marijuana
1,2018-12-30 18:51:14+00:00,Rahul from TCS grabbed the Man of the Match in the TCS-Atos match\n#puneITCup2018 #PuneSports #PrathamSports #CorporateCricket #TCS #Atos #CSR #AnnualSportsEvent #CorporateSports #PuneITCupLeagueMatches #ManOfTheMatch https://t.co/Xn9ACOxsac,rahul tc grabbed man match tcs-atos match puneitcup punesports prathamsports corporatecricket tc atos csr annualsportsevent corporatesports puneitcupleaguematches manofthematch
2,2018-12-30 18:18:52+00:00,@siasatpk @MuradSaeedPTI I'm from Karak one week before I was there in post office I shocked to see #TCS using Pakistan post for sending there couriers. @MuradSaeedPTI keep it up,karak one week post office shock see tc use pakistan post send courier keep
3,2018-12-30 17:40:00+00:00,#TCS - TCS (Tata Consultancy Services Limited) #SELL BELOW 1890 - TradingView - https://t.co/YajWHOu0qs,tc tc tata consultancy service limited sell tradingview
4,2018-12-30 17:32:09+00:00,TCS (Tata Consultancy Services Limited) #SELL BELOW 1890 - #TCS chart https://t.co/lnKukdwRG9,tc tata consultancy service limited sell tc chart
...,...,...,...
9993,2018-01-01 07:26:31+00:00,5/6 RAKESHPATEL-Q-JAN18 out the changes at https://t.co/fAN4ZYQN8x #ACCELYA #AJANTPHARM #IGL #BYKE #TCS #MONSANTO #VSTIND #MAYURUNIQ,rakeshpatel-q-jan change
9994,2018-01-01 07:23:47+00:00,1/2 RAKESHPATEL-Q-JAN18 Another new portfolio - https://t.co/fAN4ZYQN8x - Created by Rakesh-Patel #ACCELYA #AJANTPHARM #IGL #BYKE #TCS,rakeshpatel-q-jan another new portfolio
9995,2018-01-01 07:21:34+00:00,It is your opportunity to write a beautiful story for yourself.\nThe @InterviewReady_ team wishes you a very Happy New Year. #happynewyear #2018 #interviewready #TCS #careercoach https://t.co/tVJ4omi33t,opportunity write beautiful story team wish happy new year happynewyear interviewready tc careercoach
9996,2018-01-01 06:40:20+00:00,2K18 Here we are\nWe made it.\nJah mi dey grateful\n#our year of divine open doors\n#TCS @ Satellite… https://t.co/QM1DegF3r5,make jah dey grateful year divine open door tc satellite


In [ ]:
 !pip install transformers
!pip install transformers[sentencepiece]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 51.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 16.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 12.8 MB/s eta 0:00:00


In [ ]:
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
from scipy.special import softmax

In [ ]:
tweetss = pd.read_csv("/content/drive/MyDrive/major datasets /CleanedNTweets_wipro.csv")


In [ ]:
tweets1 = pd.read_csv("/content/CleanedNTweets_wipro.csv")

In [ ]:
MODEL = f"cardiffnlp/twitter-xlm-roberta-base-sentiment"

tokenizer = AutoTokenizer.from_pretrained(MODEL)
config = AutoConfig.from_pretrained(MODEL)

# TF
model = TFAutoModelForSequenceClassification.from_pretrained(MODEL)
model.save_pretrained(MODEL)
tokenizer.save_pretrained(MODEL)

All model checkpoint layers were used when initializing TFXLMRobertaForSequenceClassification.

All the layers of TFXLMRobertaForSequenceClassification were initialized from the model checkpoint at cardiffnlp/twitter-xlm-roberta-base-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFXLMRobertaForSequenceClassification for predictions without further training.


('cardiffnlp/twitter-xlm-roberta-base-sentiment/tokenizer_config.json',
 'cardiffnlp/twitter-xlm-roberta-base-sentiment/special_tokens_map.json',
 'cardiffnlp/twitter-xlm-roberta-base-sentiment/sentencepiece.bpe.model',
 'cardiffnlp/twitter-xlm-roberta-base-sentiment/added_tokens.json',
 'cardiffnlp/twitter-xlm-roberta-base-sentiment/tokenizer.json')

In [ ]:
def polarity(text):
    encoded_input = tokenizer(text, return_tensors='tf')
    output = model(encoded_input)
    scores = output[0][0].numpy()
    scores = softmax(scores)
    
    # Print labels and scores
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    l = config.id2label[ranking[0]]
    if l == "negative":
      plrty = -1
    elif l == "positive":
      plrty = 1
    else:
      plrty = 0 
    s = np.round(float(scores[ranking[0]]), 4)
    print(l, plrty)
    return (l,plrty)

In [ ]:
tweets['cleaned'].isna().sum()

0

In [ ]:
tweets['cleaned'].fillna("", inplace = True)

In [ ]:
# downloading the file after applying sentiment analysis on the current folder
tweets['label'],tweets['Polarity'] = zip(*tweets['cleaned'].apply(lambda txt:polarity(txt)))
tweets.to_csv("polarizedTweets_sample.csv",index=False)

In [ ]:
import pandas as pd
ptweets = pd.read_csv("/content/tcs_tweets_data.csv")
ptweets

,Unnamed: 0.1,Date,Tweets,cleaned,label,Polarity,Unnamed: 0
0,0,2022-12-13 22:08:20+00:00,#TECHM is one of the worst performing stock among #NIFTYIT. #TCS and #INFOSYS are trading at 15 to 18% discount from the ATH. Still techm is at -41%. https://t.co/z5Zbh0V3lC,techm one bad perform stock among niftyit tc infosys trading discount ath still techm,negative,-1,NaN
1,1,2022-12-13 21:26:59+00:00,If anyone wants to get me anything for Christmas..\n#TCS Gaming mouse pad https://t.co/YwhhSlHUeG,anyone want get anything christmas tc game mouse pad,neutral,0,NaN
2,2,2022-12-13 21:15:00+00:00,"How #TCS is using the #cloud to future-proof businesses - “That means implementing #newbusinessmodels, bringing in #agility, measuring the #businessvalue.” @TCS \n@SiliconANGLE https://t.co/Mp7naI7Wiz",tc use cloud future-proof business mean implement newbusinessmodels bring agility measure businessvalue,neutral,0,NaN
3,3,2022-12-13 20:15:26+00:00,"#TCS, #GujaratGovt Partner to Impart Digital Skills in Schools - Channel Drive\n@TCS\n@telecomdrive https://t.co/5Y8qY9bHRg",tc gujaratgovt partner impart digital skill school channel drive,neutral,0,NaN
4,4,2022-12-13 19:00:00+00:00,"@khanum_moalem_ @EleanorNorton @RepMaxineWaters @SanfordBishop @WhipClyburn Can we have wage based GC, otherwise body shops like #infosys #tcs #wipro will exploit skilled workers by making them work for low wages and illegal contract bonds #EagleAct . This will encourage skilled IT workers to compromise on low wages in return of a quick GC.",wage base otherwise body shop like infosys tc wipro exploit skilled worker make work low wage illegal contract bond eagleact encourage skilled worker compromise low wage return quick,neutral,0,NaN
...,...,...,...,...,...,...,...
66100,66100,2018-01-01 07:26:31+00:00,5/6 RAKESHPATEL-Q-JAN18 out the changes at https://t.co/fAN4ZYQN8x #ACCELYA #AJANTPHARM #IGL #BYKE #TCS #MONSANTO #VSTIND #MAYURUNIQ,rakeshpatel-q-jan change,negative,-1,4993.0
66101,66101,2018-01-01 07:23:47+00:00,1/2 RAKESHPATEL-Q-JAN18 Another new portfolio - https://t.co/fAN4ZYQN8x - Created by Rakesh-Patel #ACCELYA #AJANTPHARM #IGL #BYKE #TCS,rakeshpatel-q-jan another new portfolio,neutral,0,4994.0
66102,66102,2018-01-01 07:21:34+00:00,It is your opportunity to write a beautiful story for yourself.\nThe @InterviewReady_ team wishes you a very Happy New Year. #happynewyear #2018 #interviewready #TCS #careercoach https://t.co/tVJ4omi33t,opportunity write beautiful story team wish happy new year happynewyear interviewready tc careercoach,positive,1,4995.0
66103,66103,2018-01-01 06:40:20+00:00,2K18 Here we are\nWe made it.\nJah mi dey grateful\n#our year of divine open doors\n#TCS @ Satellite… https://t.co/QM1DegF3r5,make jah dey grateful year divine open door tc satellite,positive,1,4996.0


In [ ]:
ptweets['Polarity'].value_counts()

 0    52881
 1     7205
-1     6019
Name: Polarity, dtype: int64

In [ ]:
ptweets_df = ptweets.loc[:,["Date","Polarity"]]
ptweets_df.head()

,Date,Polarity
0,2022-12-13 22:08:20+00:00,-1
1,2022-12-13 21:26:59+00:00,0
2,2022-12-13 21:15:00+00:00,0
3,2022-12-13 20:15:26+00:00,0
4,2022-12-13 19:00:00+00:00,0


In [ ]:
ptweets.drop(57139, axis=0, inplace=True)

In [ ]:
ptweets[57139:57141]

,Unnamed: 0.1,Date,Tweets,cleaned,label,Polarity,Unnamed: 0
57140,57140,2018-11-20 11:02:48+00:00,2014.02V #TCS #CDP For Cars/Trucks #DiagnosticTools for Multi-language TCS CDP without Bluetooth\n\nhttps://t.co/on111HieBw https://t.co/Ep97l0jceY,tc cdp car truck diagnostictools multi-language tc cdp without bluetooth,neutral,0,1033.0
57141,57141,2018-11-20 11:00:48+00:00,2014.02V #TCSCDP For Cars/Trucks #TCS #CDP #Pro+ #Diagnostic Tools with Bluetooth\n\nhttps://t.co/RKVvA4l7o3 https://t.co/AWcLvqxALe,tcscdp car truck tc cdp pro diagnostic tool bluetooth,neutral,0,1034.0


In [ ]:
# Change the date format to match with the next csv file date format
ptweets_df['Date'] =pd.to_datetime(ptweets_df['Date'],infer_datetime_format=True)
ptweets_df['Date'] =pd.to_datetime(ptweets_df['Date'].dt.strftime("%m/%d/%y"))

# Aggregate the tweets polarization by avergae, sum and counts 
Pol_df = pd.DataFrame(ptweets_df.groupby('Date')['Polarity'].mean())
Pol_df.rename(columns={"Polarity":"P_mean"},inplace=True)
Pol_df['P_sum'] = ptweets_df.groupby('Date')['Polarity'].sum()
Pol_df['twt_count'] = ptweets_df.groupby('Date')['Polarity'].count()
Pol_df.head()

In [ ]:
Pol_df

Finance data

In [ ]:
!pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from datetime import datetime

In [ ]:
#Extracting stock data 

import yfinance as yf
stock_data = yf.download('TCS.NS', start="2018-01-01", end="2022-12-31")

[*********************100%***********************]  1 of 1 completed


In [ ]:
stock_data.to_csv('stock_data.csv', index=True)

In [ ]:
stock_data = pd.read_csv('stock_data.csv')

In [ ]:
stock_data

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-01-01,1341.150024,1347.400024,1317.500000,1322.800049,1184.243530,1351760
1,2018-01-02,1330.000000,1334.800049,1310.099976,1315.599976,1177.797607,1920290
2,2018-01-03,1316.000000,1334.500000,1315.599976,1319.324951,1181.132446,1257120
3,2018-01-04,1325.000000,1331.000000,1320.000000,1328.550049,1189.391235,913082
4,2018-01-05,1325.000000,1349.750000,1325.000000,1344.599976,1203.760132,1153706
...,...,...,...,...,...,...,...
1231,2022-12-26,3228.350098,3272.000000,3225.050049,3252.899902,3188.315186,870157
1232,2022-12-27,3269.199951,3273.800049,3231.500000,3259.500000,3194.784424,835883
1233,2022-12-28,3249.800049,3266.100098,3226.000000,3257.100098,3192.432129,910795
1234,2022-12-29,3231.100098,3271.500000,3228.000000,3268.750000,3203.850586,1037927


In [ ]:
#Converting date column type to datetime from object
stock_data['Date'] = pd.to_datetime(stock_data.Date,format='%Y-%m-%d')

In [ ]:
stock_data['Close'].isna().sum()

0

In [ ]:
stock_data.set_index("Date")

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-01,1341.150024,1347.400024,1317.500000,1322.800049,1184.243530,1351760
2018-01-02,1330.000000,1334.800049,1310.099976,1315.599976,1177.797607,1920290
2018-01-03,1316.000000,1334.500000,1315.599976,1319.324951,1181.132446,1257120
2018-01-04,1325.000000,1331.000000,1320.000000,1328.550049,1189.391235,913082
2018-01-05,1325.000000,1349.750000,1325.000000,1344.599976,1203.760132,1153706
...,...,...,...,...,...,...
2022-12-26,3228.350098,3272.000000,3225.050049,3252.899902,3188.315186,870157
2022-12-27,3269.199951,3273.800049,3231.500000,3259.500000,3194.784424,835883
2022-12-28,3249.800049,3266.100098,3226.000000,3257.100098,3192.432129,910795


In [ ]:
data = stock_data.join(Pol_df,on='Date',how="inner")

In [ ]:
data.to_csv("TCS_Data.csv",index=False)

In [ ]:
infosys_data = pd.read_csv('TCS_Data.csv')
infosys_data

,Date,Open,High,Low,Close,Adj Close,Volume,P_mean,P_sum,twt_count
0,2019-12-16,2096.000000,2136.000000,2080.300049,2126.750000,1995.699341,5574177,0.090909,5,55
1,2019-12-17,2126.500000,2169.000000,2119.000000,2164.949951,2031.545166,6059673,-0.090909,-5,55
2,2019-12-18,2174.000000,2203.449951,2153.000000,2167.750000,2034.173340,8251081,-0.037500,-3,80
3,2019-12-19,2179.800049,2236.500000,2167.550049,2229.050049,2091.695557,6251505,0.090909,4,44
4,2019-12-20,2220.000000,2246.699951,2208.250000,2222.899902,2085.924561,5548338,0.069767,3,43
...,...,...,...,...,...,...,...,...,...,...
740,2022-12-07,3368.300049,3393.699951,3354.000000,3380.600098,3313.479980,1167505,0.116667,7,60
741,2022-12-08,3360.000000,3380.600098,3339.449951,3350.550049,3284.026611,1285619,-0.032787,-2,61
742,2022-12-09,3374.100098,3374.100098,3276.350098,3292.750000,3227.374023,1797896,-0.015873,-1,63
743,2022-12-12,3255.500000,3296.000000,3236.100098,3286.449951,3221.199219,1527610,-0.105263,-6,57
